# Part 15: Beacon Exclusion Zone

## Setup

In [1]:
data = [line for line in open("./inputs/day15.txt", "r").read().splitlines()]

## Helpers
Parsing the four values by finding indices by '='.<br>
<br>
merge_ranges just normalizes from-to ranges, i.e [[0, 5],[1, 6]] => [[0, 6]]<br>
line_coverage checks each line for sensor coverage by including sensors within manhattan distance<br>
Part 2 added an optional "limit" argument.<br>

In [2]:
def manhattan_distance(s, p):
  return abs(s[0]-p[0]) + abs(s[1]-p[1])

sensors = []
distances = []
beacons = set()

i = 0
for l in data:
  indices = [i for i, c in enumerate(l) if c == '=']
  sensor_x, sensor_y, beacon_x, beacon_y = [
      int(l[indices[0]+1:].split(',')[0]),
      int(l[indices[1]+1:].split(':')[0]),
      int(l[indices[2]+1:].split(',')[0]),
      int(l[indices[3]+1:])]
  man_dist = manhattan_distance((sensor_x, sensor_y), (beacon_x, beacon_y))

  sensors.append((sensor_x, sensor_y))
  distances.append(man_dist)
  beacons.add((beacon_x, beacon_y))

def merge_ranges(ranges):
  start = sorted(ranges)
  stack = [start[0]]

  for cur in start[1:]:
    pre = stack[-1]
    if pre[0] <= cur[0] <= pre[1]:
      pre[1] = max(pre[1], cur[1])
    else:
      stack.append(cur)

  return sorted(stack)

def line_coverage(y, limit=None):
  coverage = []
  for i, c in enumerate(sensors):
    if y-distances[i] <= c[1] <= y+distances[i]:
      offset = distances[i]-abs(y-c[1])

      if limit:
        coverage.append([max(c[0]-offset,limit[0]), min(c[0]+offset,limit[1])])
      else:
        coverage.append([c[0]-offset, c[0]+offset])

  return merge_ranges(coverage)

## Part 1
Line coverage for one row, and substracting the number of beacons on said row.

In [3]:
y = 2_000_000
coverage = line_coverage(y)
part1 = sum([len(range(c[0], c[1]+1)) for c in coverage])
part1 -= len([b for b in beacons if b[1] == y])

print(part1)

5125700


## Part 2
We're iterating all 4 million rows to see if coverage has holes in it.<br>
A soon as we see a hole in the coverage, we're exiting.<br>

In [4]:
limit = [0, 4_000_000]
new_beacon = 0, 0

for y in range(limit[0], limit[1]+1):
  coverage = line_coverage(y, limit)
  if len(coverage) > 1:
    new_beacon = coverage[0][1] + 1, y
    break
  elif coverage[0][0] != limit[0]:
    new_beacon = limit[0], y
    break
  elif coverage[0][1] != limit[1]:
    new_beacon = limit[1], y
    break

print(new_beacon[0]*limit[1]+new_beacon[1])

11379394658764
